# Building a model to predict guest ridership

### Given this data, do we improve the fit of our model (R^2)?

In [52]:
# This should all be familiar by now...
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import linear_model, metrics

import statsmodels.api as sm

%matplotlib inline

bike_data = pd.read_csv('https://github.com/ga-students/DAT-NYC-37/raw/master/lessons/lesson-07/assets/dataset/bikeshare.csv')
bike_data.head(3)

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32


#### The following function was provided for you, however you should be familiar with each step of the function

In [53]:
from sklearn import feature_selection, linear_model

# From last class...
def get_linear_model_metrics(X, y):
    # 1. Define and configure the model:
    model = linear_model.LinearRegression()
    f, pvals = feature_selection.f_regression(X, y)

    # 2. Fit the model
    model.fit(X, y)

    # 3. Evaluate & score the model
    residuals = (y - model.predict(X)).values

    print 'P values:', pvals
    print 'Coefficients:', model.coef_
    print 'y-intercept:', model.intercept_
    print 'R-Squared:', model.score(X,y)
    print
    
    return model

### Our starting point:

*(This was done in class and provided for you)*

- Select `atemp`, `hum`, and `weather` as features
- `weather` is a 4 level factor (1, 2, 3, 4), so we converted it to a dummy variable and removed weather_4
- `casual` Was defined as our target variable. (The number of riders in a given hour of the day)

### The starter code you were given:

In [54]:
weather = pd.get_dummies(bike_data.weathersit)

#create new names for our new dummy variables
weather.columns = ['weather_' + str(i) for i in weather.columns]

#join those new variables back into the larger dataset
bikemodel_data = bike_data.join(weather)

columns_to_keep = ['hum', 'temp', 'weather_1', 'weather_2', 'weather_3']

X = bikemodel_data[columns_to_keep]  # Our input matrix
y = bikemodel_data['casual'] # our target

get_linear_model_metrics(X, y);

P values: [  0.00000000e+00   0.00000000e+00   3.75616929e-73   3.43170021e-22
   1.57718666e-55]
Coefficients: [ -84.01121684  112.68901765  -24.68489063  -21.00314494  -21.71893628]
y-intercept: 55.8412915804
R-Squared: 0.311934605989



## Starter code result

So far, our baseline `R^2` is **`0.311`**. How much can we improve upon it?

### A decent starting point:
Intuitively, it seems like `workingday` would be a good a feature to choose since we'd expect more riders on such days.

In [55]:
columns_to_keep = ['hum', 'temp', 'weather_1', 'weather_2', 'weather_3', 'workingday']

X = bikemodel_data[columns_to_keep]  # Our input matrix
y = bikemodel_data['casual'] # our target

get_linear_model_metrics(X, y);

P values: [  0.00000000e+00   0.00000000e+00   3.75616929e-73   3.43170021e-22
   1.57718666e-55   0.00000000e+00]
Coefficients: [ -84.14725133  117.70452779  -26.28394685  -21.25298575  -20.81182552
  -34.25197022]
y-intercept: 77.85987364
R-Squared: 0.415888672027



#### Great! By adding one feature, we've improved our R^2 to 0.4158

### My Approach:

In the starter code we "One-Hot" encoded (i.e. the "dummy variables") the `weathersit` categorical variable. However, this isn't the only categorical variable.

Other categorical variables:
    - season
    - yr
    - mnth
    - weekday
    - hr
    
Let's try encoding each of these into dummy variables, as we did with `weathersit` above

*Question: Why am I One-Hot encoding `yr`, `mnth`, `weekday`, as if they are categorical?*

In [56]:
# Note: Same exact process as before, but now for the `season`, `weekday`, `hr`, `month` columns
season = pd.get_dummies(bike_data.season)
season.columns = ['season_' + str(i) for i in season.columns]

weekday = pd.get_dummies(bike_data.weekday)
weekday.columns = ['weekday_' + str(i) for i in weekday.columns]

hr = pd.get_dummies(bike_data.hr)
hr.columns = ['hr_' + str(i) for i in hr.columns]

month = pd.get_dummies(bike_data.mnth)
month.columns = ['month_' + str(i) for i in month.columns]

#join those new variables back into the larger dataset
bikemodel_data = pd.concat([bike_data, weather, season, month, hr, weekday], axis=1);

Now that I've encoded my categorical variables and added the dummy variables to a copy of my original dataframe (`bikemodel_data`), I'm going to exclude the following columns from the original dataframe to produce the vector X:

<div style="color: red;">
<h3>Important Note:</h3>

<p>
We should always discard one of the dummy variables? Why? Because that's all we need to encode the levels for each category. For instance, the categorical variable `weathersit` had 4 factors: 1, 2, 3, 4 but if we know 3 out of 4 of these values, then the remaining value can be deduced. For example, if I tell you it can ONLY be sunny, rainy, windy, or snowy on a given day and I also tell you that tomorrow's not going to be sunny, rainy, or windy, then what kind of day will it be tomorrow?
</p>
</div>

If this is confusing, think about why we only needed one dummy variable for a boolean variable Size (IsLarge), not two dummy variables (IsSmall and IsLarge).

**<p style="color: darkorange;"> !! Key takeaway: If you have a categorical feature with k levels, you create k-1 dummy variables !!</p>**

**Outcome variables:**

    'casual',       # This is our target variable, which is an outcome and therefore not an independent variable
    'registered',   # Also an outcome, not an independent variable.
    'cnt'           # Also an outcome, not an independent variable (actually casual + registered)
    
**Index/id variables:**

    'instant',    # This is just the row number
    'dteday',     # This is a date column, and is neither categorical nor numerical. Furthermore, this information 
                  #  is already present in the `yr`, `mnth`, `weekday` columns
    
**Redundant/correlated variables**

*Remember! When we create dummy variables we must remove one of them to prevent redundancy in our input data.*

    'temp'        # Strongly correlated with atemp, so we only should keep atemp or temp (discussed in class)
    'weather_4'
    'season_4'   
    'weekday_6'
    'month_11'
    'hr_23'
    
**Encoded variables**

*Think about why we need to remove the following variables as well...*

    'weathersit', 
    'season', 
    'weekday',
    'workingday',
    'hr',
    'mnth'

In [59]:
# Define our input variable, X by excluding the columns listed above from our dataset:
columns_to_drop = [
        'instant',
        'dteday',
        'weather_4',
        'season_4',
        'weekday_6',
        'month_11',
        'hr_0',
        'weathersit', 
        'season', 
        'weekday',
        'workingday',
        'hr',
        'mnth',
        'casual',
        'registered', 
        'temp', 
        'cnt'
    ]

X = bikemodel_data.drop(columns_to_drop, axis=1)

# We've only kept these columns
print "COLUMNS: " + " ".join(X.columns.values)

get_linear_model_metrics(X, y);

COLUMNS: yr holiday atemp hum windspeed weather_1 weather_2 weather_3 season_1 season_2 season_3 month_1 month_2 month_3 month_4 month_5 month_6 month_7 month_8 month_9 month_10 month_12 hr_1 hr_2 hr_3 hr_4 hr_5 hr_6 hr_7 hr_8 hr_9 hr_10 hr_11 hr_12 hr_13 hr_14 hr_15 hr_16 hr_17 hr_18 hr_19 hr_20 hr_21 hr_22 hr_23 weekday_0 weekday_1 weekday_2 weekday_3 weekday_4 weekday_5
P values: [  8.09908774e-080   3.15814032e-005   0.00000000e+000   0.00000000e+000
   8.66781628e-033   3.75616929e-073   3.43170021e-022   1.57718666e-055
   7.90819136e-239   1.70941845e-060   1.20544330e-119   7.20393520e-107
   6.04893370e-081   7.48894210e-006   9.89721401e-008   2.10345324e-034
   2.02298294e-036   1.86526710e-043   6.89088813e-027   1.52784974e-026
   8.70854193e-006   8.98402890e-067   6.64996464e-060   3.26347943e-066
   3.30809960e-073   9.64851198e-080   2.34162115e-081   8.63420127e-070
   2.84021955e-043   5.03259555e-015   7.51024245e-003   1.56865913e-009
   9.55402696e-041   5.4256314

#### Our R^2 increased to `0.588`!

### Note the following conditions hold:

- We have small p-values for all  features
- None of our coefficients have extremely large values nor are they close to zero.

#### However! We haven't yet validated our model on data we haven't seen.

### Questions:

1. Our R^2 ALWAYS increases when we add an input feature. Why? Is this necessarily a good thing? Why or why not?
2. Rather than dropping our original `season` column, try keeping it in the input dataframe X by removing `'season'` from our `columns_to_exclude` list. What happens to our coefficients and why does this happen?
3. Do the same thing for the `workingday`. Again explain what's causing any anomalies you see in our input dataframe X.
4. It can be argued that `yr`, `mnth`, `weekday`, and `hr` are numerical variables, not categorical. Why did we treat them as categorical and are there any drawbacks to this approach?
5. Our input vector `X` has 51 columns! Suppose we could only keep 6 columns in our input. Which columns would you keep and why?